In [1]:
import os, time, sys,shutil
import pandas as pd
from digital_land.collection import Collection

from functions import run_endpoint_workflow
from sqlite_query_functions import DatasetSqlite
from convert_functions import convert_resource




/Users/jimmytidey/Projects/jupyter-analysis/venv/lib/python3.10/site-packages/pyproj/transformer.py:207: UserWarning: Best transformation is not available due to missing Grid(short_name=uk_os_OSTN15_NTv2_OSGBtoETRS.tif, full_name=, package_name=, url=https://cdn.proj.org/uk_os_OSTN15_NTv2_OSGBtoETRS.tif, direct_download=True, open_license=True, available=False)
  super().__init__(


In [25]:
def conservation_area_checker(url):
    collection_name = 'listed-building-collection'
    dataset = 'listed-building-outline'
    organisation = 'local-authority-eng:CAM'
    endpoint_url = url
    plugin = None
    additional_column_mappings=None
    additional_concats=None
    data_dir = 'data/endpoint_checker_' + str(int(time.time()*10000)) 

    workflow_state = run_endpoint_workflow(
        collection_name,
        dataset,
        organisation,
        endpoint_url,
        plugin,
        data_dir,
        additional_col_mappings=additional_column_mappings,
        additional_concats=additional_concats
    )

    if(workflow_state):
        checker_state = 'Data checker failed - ensure you have a valid CSV, GeoJSON, GML, or Geopackage file'
        ret = {'checker_state': checker_state, 'issues': []}
        shutil.rmtree(data_dir)
        return ret   

    collection = Collection(os.path.join(data_dir,'collection'))
    collection.load(directory=os.path.join(data_dir,'collection'))
    logs = collection.log.entries
    logs = pd.DataFrame.from_records(logs)
    display(logs)
    http_status = logs['status'].values[0]

    if(http_status != '200'): 
        checker_state = "Could not access URL"
        ret = {'checker_state': checker_state, 'issues': []}
        shutil.rmtree(data_dir)
        return ret   

    try: 
        dataset_db = DatasetSqlite(os.path.join(data_dir,'dataset',f'{dataset}.sqlite3'))
        results = dataset_db.get_issues()
        display(results)
        row_count = results['line_number'].drop_duplicates().size

        if(row_count == 0):
            checker_state = "No rows were processed, check you have created a valid file."
        else:
            checker_state = f"{row_count} rows have been processed"

        issues = results.loc[(results.issue_type != 'default-value')]

    except:
        issues = pd.DataFrame()

   
    ret = {'checker_state': checker_state, 'issues': issues.to_dict('records')}
    
    shutil.rmtree(data_dir)

    return ret


conservation_area_checker('https://raw.githubusercontent.com/jimmytidey/jupyter-analysis/feature-endpoint-checker-api/endpoint_checker/examples/conservation_area_duplicate_references.csv')


HTTP Error 404: Not Found
HTTP Error 404: Not Found
data/endpoint_checker_16967780703596/collection
[{'resource': '506d93ab11fc65e8c791f14198d8a250f7656f5c5865828983d2490ea4412a48', 'bytes': '397', 'endpoints': '48a28a3c63ba818b26d45c336ed9133945e50dc3efae76c6435e6ecbb424884b', 'organisations': 'local-authority-eng:CAM', 'datasets': 'listed-building-outline', 'start-date': '2023-10-08', 'end-date': ''}]


,bytes,content-type,elapsed,endpoint,resource,status,entry-date,start-date,end-date,exception
0,397,text/plain; charset=utf-8,0.221,48a28a3c63ba818b26d45c336ed9133945e50dc3efae76...,506d93ab11fc65e8c791f14198d8a250f7656f5c586582...,200,2023-10-08T08:28:41.127595,,,


,end_date,entry_date,entry_number,field,issue_type,line_number,dataset,resource,start_date,value
0,,,1,organisation,default-value,2,listed-building-outline,506d93ab11fc65e8c791f14198d8a250f7656f5c586582...,,local-authority-eng:CAM
1,,,1,entry-date,default-value,2,listed-building-outline,506d93ab11fc65e8c791f14198d8a250f7656f5c586582...,,2023-10-08
2,,,2,organisation,default-value,3,listed-building-outline,506d93ab11fc65e8c791f14198d8a250f7656f5c586582...,,local-authority-eng:CAM
3,,,2,entry-date,default-value,3,listed-building-outline,506d93ab11fc65e8c791f14198d8a250f7656f5c586582...,,2023-10-08


{'checker_state': '2 rows have been processed', 'issues': []}